# A Bottom-up Approach to Complex Rhythms

In [1]:
from abjad import *

# Tuplets

The Tuplet class provides an interface for working with complex rhythms in a bottom-up way.

In [2]:
tuplet = Tuplet(Multiplier(2, 3), "c'8 d'8 e'8")

In [3]:
show(tuplet)

You can also pack existing leaves into a Tuplet instance.

In [4]:
leaves = [Note("fs'8"), Note("g'8"), Rest('r8')]

In [5]:
tuplet = Tuplet(Multiplier(2, 3), leaves)

In [6]:
show(tuplet)

Tuplet instances all have a multiplier and components.

In [7]:
tuplet

Tuplet(Multiplier(2, 3), "fs'8 g'8 r8")

## Understanding Augmentation and Diminution

Remember that any tuplet can be represented as an augmentation or a diminution relative to the written notes' default values. Our example tuplet's multiplier of (2,3) for three eighth notes means that each written eighth note lasts for 2/3rds its written value. Because the original durations have been reduced, this is a diminution:

In [8]:
tuplet.is_diminution

True

A tuplet with a multiplier greater than 1, on the other hand, would be an augmentation:

In [11]:
tuplet = Tuplet((4,3), "fs'16 g'16 r16")

In [12]:
show(tuplet)


This last tuplet is an augmentation in which each of the written sixteenth notes lasts for 4/3rds of its written duration. The sounding result would be identical, and these are just two ways of writing the same thing, the former of which happens to be conventional.

Remember that object-oriented programming gives us objects with characteristics and behaviors. We can use the dot-chaining syntax to read and write the tuplet's multiplier attribute:

In [18]:
tuplet = Tuplet(Multiplier(2, 3), "fs'8 g' r8")

In [19]:
tuplet.multiplier

Multiplier(2, 3)

In [20]:
tuplet.multiplier = Multiplier(4, 5)

In [21]:
tuplet

Tuplet(Multiplier(4, 5), "fs'8 g'8 r8")

In [22]:
show(tuplet)

## Adding Leaves

The multiplier is a reasonable characteristic for our tuplet to have, but what about behaviors? Well, you probably want to be able to build up tuplets by adding leaves to them, one or several a time. The append method adds leaves to the end of a tuplet (and to any Python list), one leaf at a time:

In [23]:
tuplet.append(Note("e'4."))

In [24]:
show(tuplet)

...or using a LilyPond string:

In [25]:
tuplet.append("bf8")

In [26]:
show(tuplet)

Likewise, the extend method adds two or more leaves at a time:

In [27]:
notes = [Note("fs'32"), Note("e'32"), Note("d'32"), Rest((1, 32))]

In [28]:
tuplet.extend(notes)

In [36]:
show(tuplet)

And you can use a LilyPond string with extend, too:

In [30]:
tuplet.extend("gs'8 a8")

In [35]:
show(tuplet)

## Removing Leaves

You can remove tuplet components by reference using the remove method:

In [ ]:
tuplet.remove(tuplet[3])

In [ ]:
show(tuplet)

If you want to remove a component by index and then keep it to do something else with it, use the pop method instead of remove:

In [ ]:
popped = tuplet.pop(2)

In [ ]:
show(tuplet)

## Indexing Leaves

Tuplets support indexing, if you'd like to do something to the nth component of a tuplet:

In [ ]:
tuplet[1]

If you've added an existing list to a tuplet's components, and you'd like to see where a component of that list is in the tuplet, you can use the tuplet's index method - in our case, we'll use our notes list from above:

In [32]:
notes

[Note("fs'32"), Note("e'32"), Note("d'32"), Rest('r32')]

In [33]:
notes[1]

Note("e'32")

In [34]:
tuplet.index(notes[1])

6

The second item in our notes list is the seventh component in our tuplet.

show(tuplet)

## Making Tuplets from Durations and Ratios

The Tuplet class also provides a method for constructing a tuplet from a duration and a ratio, where the ratio is a list of integers, the sum of which determines the number of equally spaced pulses within the duration:

In [76]:
tuplet = Tuplet.from_duration_and_ratio((1,4), [1,3,1])

In [77]:
staff = scoretools.Staff([tuplet], context_name='RhythmicStaff')

In [78]:
show(staff)

This might not seem like much, but we can write functions that use these kinds of basic functionalities to do more complicated things, like we'll do in this next example, taken from a real score:

# Brian Ferneyhough - Unsichtbare Farben 

Mikhial Malt analyzes the rhythmic materials of Ferneyhough’s solo violin composition, Unsichtbare Farben, in The OM Composer’s Book 2.

Malt explains that Ferneyhough used OpenMusic to create an “exhaustive catalogue of rhythmic cells” such that:

Each cell consists of the same duration divided into two pulses related by a durational proportion ranging from 1:1 to 1:11.

The second pulse is then subdivided successively into 1, 2, 3, 4, 5 and 6 equal parts.

Let’s recreate Malt’s results in Abjad.

## The Proportions 

We use a list comprehension to describe a list of (1,n) tuples, each of which will describe the durational proportion between a cell's first and second pulse:

In [ ]:
proportions = [(1, n) for n in range(1, 11 + 1)]

In [ ]:
proportions

## The Transforms

Next we’ll show how to divide a quarter note into various ratios, and then divide the final logical tie of the resulting tuplet into yet another ratio:

In [37]:
def make_nested_tuplet(
    tuplet_duration,
    outer_tuplet_proportions,
    inner_tuplet_subdivision_count,
    ):
    r'''Makes nested tuplet.
    '''

    outer_tuplet = Tuplet.from_duration_and_ratio(
        tuplet_duration, outer_tuplet_proportions)
    inner_tuplet_proportions = inner_tuplet_subdivision_count * [1]
    last_leaf = outer_tuplet.select_leaves()[-1]
    right_logical_tie = inspect_(last_leaf).get_logical_tie()
    right_logical_tie.to_tuplet(inner_tuplet_proportions)
    return outer_tuplet

And of course it's easier to see what this function does with an example of use:

In [38]:
tuplet = make_nested_tuplet(Duration(1, 4), (1, 1), 5)

In [39]:
staff = scoretools.Staff([tuplet], context_name='RhythmicStaff')

In [40]:
show(staff)

We see that a duration of a quarter note (the first argument) has been divided into two pulses with a durational proportion of 1:1 (second argument), the second pulse of which has then been divided into five equally spaced parts (the third argument). Try changing the arguments and see what happens.

In [ ]:
tuplet = make_nested_tuplet(Duration(1, 4), (2, 1), 5)

In [ ]:
staff = scoretools.Staff([tuplet], context_name='RhythmicStaff')

In [ ]:
show(staff)

In [ ]:
tuplet = make_nested_tuplet(Duration(1, 4), (3, 1), 5)

In [ ]:
staff = scoretools.Staff([tuplet], context_name='RhythmicStaff')

In [ ]:
show(staff)

### Logical Ties Solve the Problem of Five

A logical tie is a selection of notes or chords connected by ties. It lets us talk about a notated rhythm of 5/16, for example, which can not be expressed with only a single leaf.

Note how we can divide a tuplet whose outer proportions are 3/5, where the second logical tie requires two notes to express the 5/16 duration:

In [41]:
outer_tuplet = Tuplet.from_duration_and_ratio(Duration(1, 4), (3, 5))

In [42]:
staff = scoretools.Staff([outer_tuplet], context_name='RhythmicStaff')

In [43]:
show(staff)

In [44]:
subdivided_tuplet = make_nested_tuplet(Duration(1, 4), (3, 5), 3)

In [45]:
staff = scoretools.Staff([subdivided_tuplet], context_name='RhythmicStaff')

In [46]:
show(staff)

Do you see which objects and methods in our make_nested_tuplet function convert a logical tie into a tuplet?

## The Rhythms

Now that we know how to make the basic building block, let’s make a lot of tuplets all at once.

We’ll set the duration of each tuplet equal to a quarter note:

In [53]:
duration = Fraction(1,4)

Reusing our make_nested_tuplet function, we make one row of rhythms, with the last logical tie increasingly subdivided:

In [54]:
def make_row_of_nested_tuplets(
    tuplet_duration,
    outer_tuplet_proportions,
    column_count,
    ):
    r'''Makes row of nested tuplets.
    '''

    assert 0 < column_count
    row_of_nested_tuplets = []
    for n in range(column_count):
        inner_tuplet_subdivision_count = n + 1
        nested_tuplet = make_nested_tuplet(
            tuplet_duration,
            outer_tuplet_proportions,
            inner_tuplet_subdivision_count,
            )
        row_of_nested_tuplets.append(nested_tuplet)
    return row_of_nested_tuplets

In [55]:
tuplets = make_row_of_nested_tuplets(duration, (2, 1), 6)

In [56]:
staff = scoretools.Staff(tuplets, context_name='RhythmicStaff')

In [57]:
show(staff)

If we can make one single row of rhythms, we can make many rows of rhythms. We reuse this last function to make another function:

In [60]:
def make_rows_of_nested_tuplets(tuplet_duration, row_count, column_count):
    r'''Makes rows of nested tuplets.
    '''

    assert 0 < row_count
    rows_of_nested_tuplets = []
    for n in range(row_count):
        outer_tuplet_proportions = (1, n + 1)
        row_of_nested_tuplets = make_row_of_nested_tuplets(
            tuplet_duration, outer_tuplet_proportions, column_count)
        rows_of_nested_tuplets.append(row_of_nested_tuplets)
    return rows_of_nested_tuplets

In [61]:
score = Score()

In [62]:
for tuplet_row in make_rows_of_nested_tuplets(duration, 4, 6):
    score.append(scoretools.Staff(tuplet_row, context_name='RhythmicStaff'))

In [63]:
show(score)

This example illustrates how simpler bottom-up rhythmic construction might be abstracted to explore the potential of a bottom-up rhythmic idea.